In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
import re
import string 
import numpy as np

C:\Users\Saiful\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Saiful\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\Saiful\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/main/tweet_emotions.csv').drop(columns=['tweet_id'])
df.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [3]:
df['sentiment'].value_counts()

sentiment
neutral       8638
worry         8459
happiness     5209
sadness       5165
love          3842
surprise      2187
fun           1776
relief        1526
hate          1323
empty          827
enthusiasm     759
boredom        179
anger          110
Name: count, dtype: int64

In [4]:
x = df['sentiment'].isin(['happiness','sadness'])
df = df[x]

In [5]:
df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})
df.head()

,sentiment,content
1,0,Layin n bed with a headache ughhhh...waitin o...
2,0,Funeral ceremony...gloomy friday...
6,0,"I should be sleep, but im not! thinking about ..."
8,0,@charviray Charlene my love. I miss you
9,0,@kelcouch I'm sorry at least it's Friday?


In [6]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Saiful\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Saiful\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['content'] = df['content'].apply(lower_case)
        df['content'] = df['content'].apply(remove_stop_words)
        df['content'] = df['content'].apply(removing_numbers)
        df['content'] = df['content'].apply(removing_punctuations)
        df['content'] = df['content'].apply(removing_urls)
        df['content'] = df['content'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [8]:
df = normalize_text(df)
df.head()

,sentiment,content
1,0,layin n bed headache ughhhh waitin call
2,0,funeral ceremony gloomy friday
6,0,sleep im not thinking old friend want married ...
8,0,charviray charlene love miss
9,0,kelcouch sorry least friday


In [9]:
vectorizer = CountVectorizer(max_features=1000)
X = vectorizer.fit_transform(df['content'])
y = df['sentiment']

In [10]:
X_train, X_test, y_train,y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [11]:
import dagshub
mlflow.set_tracking_uri("https://dagshub.com/Saiful0044/Analyzing-Emotional-Sentiment-in-Tweets-Using-Machine-Learning.mlflow")

dagshub.init(repo_owner='Saiful0044', repo_name='Analyzing-Emotional-Sentiment-in-Tweets-Using-Machine-Learning', mlflow=True)

mlflow.set_experiment("Logistic Regression Baseline")

Accessing as Saiful0044

Initialized MLflow to track repo "Saiful0044/Analyzing-Emotional-Sentiment-in-Tweets-Using-Machine-Learning"

Repository Saiful0044/Analyzing-Emotional-Sentiment-in-Tweets-Using-Machine-Learning initialized!

<Experiment: artifact_location='mlflow-artifacts:/71cf18d578854fb49c39799fb65a61e8', creation_time=1749970494683, experiment_id='0', last_update_time=1749970494683, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [12]:
with mlflow.start_run():
    # log preprocessing parameters
    mlflow.log_param('vectorizer', 'Bag of Words')
    mlflow.log_param('num_features', 1000)
    mlflow.log_param('test_size', 0.2)

    # model building and training
    model = LogisticRegression()
    model.fit(X_train,y_train)

    # log model parameters
    mlflow.log_param('model', 'Logistic Regression')

    # model evaluation
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test,y_pred)
    precision = precision_score(y_test,y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # log evaluation metrics
    mlflow.log_metric('accuracy', accuracy)
    mlflow.log_metric('precision', precision)
    mlflow.log_metric('recall', recall)
    mlflow.log_metric('f1_score', f1)


    # log model
    mlflow.sklearn.log_model(model, 'model')

    # save and log the notebook
    import os
    notebook_path = 'exp1_baseline_model.ipynb'
    os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
    mlflow.log_artifact(notebook_path)

    # print the results for verification
    print('accuracy: ', accuracy)
    print('precision: ', precision)
    print('recall: ', recall)
    print('f1 score: ', f1)

2025/06/16 07:43:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


accuracy:  0.7763855421686747
precision:  0.7677356656948494
recall:  0.7783251231527094
f1 score:  0.7729941291585128


2025/06/16 10:04:47 INFO mlflow.tracking._tracking_service.client: 🏃 View run rogue-sloth-703 at: https://dagshub.com/Saiful0044/Analyzing-Emotional-Sentiment-in-Tweets-Using-Machine-Learning.mlflow/#/experiments/0/runs/3b9079f028724285a5b5cc054e2abeaa.
2025/06/16 10:04:47 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/Saiful0044/Analyzing-Emotional-Sentiment-in-Tweets-Using-Machine-Learning.mlflow/#/experiments/0.
